## Обучение генеративной трансформерной модели с помощью `transformers`

Для 

In [21]:
%pip install transformers, tokenizers

Defaulting to user installation because normal site-packages is not writeable
ERROR: Invalid requirement: 'transformers,'

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
urls = ['http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0039.shtml',
        'http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0040.shtml',
        'http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0050.shtml',
        'http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0060.shtml',
        'http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0070.shtml',
        'http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0080.shtml',
        'http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0090.shtml',
        'http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_1860_dekabristy.shtml']

In [14]:
import requests
import re, html

def download(url):
    return requests.get(url).text

# code borrowed from here: https://github.com/pallets/markupsafe/blob/0.23/markupsafe/__init__.py#L21
striptags_re = re.compile(r'(<!--.*?-->|<[^>]*>)')
entity_re = re.compile(r'&([^;]+);')

def to_text(s):
    return html.unescape(striptags_re.sub('',s))

def beautify(s):
    lines = [ x.strip() for x in s.split('\n') if x.strip()!=""]
    for i in range(min(100,len(lines))):
        if lines[i]=='-->':
            break
    return '\n'.join(lines[i+1:] if i<100 else lines)
    
with open('dataset.txt','w',encoding='utf-8') as f:
    for u in urls:
        text = beautify(to_text(download(u)))
        f.write(text+'\n\n')
        

In [22]:
import transformers as tr
import tokenizers as tok

In [38]:
tokenizer = tok.Tokenizer(tok.models.BPE(unk_token='[UNK]'))
tokenizer.pre_tokenizer = tok.pre_tokenizers.Whitespace()
trainer = tok.trainers.BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer.train(['dataset.txt'],trainer)

In [41]:
tokenizer.encode('Иван Сигизмундович подошел к окну и закашлялся. Вечерело.').tokens

['Иван',
 'С',
 'иг',
 'изму',
 'н',
 'до',
 'вич',
 'подошел',
 'к',
 'окну',
 'и',
 'закашлялся',
 '.',
 'Вечер',
 'ело',
 '.']

In [42]:
tokenizer.post_processor = tok.processors.TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", tokenizer.token_to_id("[CLS]")),
        ("[SEP]", tokenizer.token_to_id("[SEP]")),
    ],
)

In [44]:
tokenizer.encode('Иван Сигизмундович подошел к окну и закашлялся.','Вечерело.').tokens

['[CLS]',
 'Иван',
 'С',
 'иг',
 'изму',
 'н',
 'до',
 'вич',
 'подошел',
 'к',
 'окну',
 'и',
 'закашлялся',
 '.',
 '[SEP]',
 'Вечер',
 'ело',
 '.',
 '[SEP]']

In [51]:
vocab = tokenizer.get_vocab()
len(vocab)

30000

In [70]:
config = tr.GPT2Config(vocab_size=len(vocab))
gpt = tr.GPT2LMHeadModel(config)

In [74]:
ttokenizer = tr.PreTrainedTokenizerFast(tokenizer_object=tokenizer)
#ttokenizer=tr.AutoTokenizer.from_pretrained("openai-gpt")
res = gpt.generate(**ttokenizer("[CLS]Мне нравится ",return_tensors="pt"),max_new_tokens=50)
ttokenizer.decode(res[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'[CLS] [CLS] Мне нравится [SEP] удары удары удары зю зю зю зю зю размахивая размахивая размахивая размахивая размахивая размахивая размахивая размахивая размахивая размахивая размахивая размахивая тяжелым тяжелым тяжелым тяжелым тяжелым велите souff souff souff souff souff souff souff souff souff souff souff souff souff вающую вающую вающую вающую вающую вающую черную черную черную черную евский'